In [1]:
# For reading credentials from the .env file
import os
from dotenv import load_dotenv
import pandas as pd

from langchain.document_loaders import PyPDFLoader, DataFrameLoader
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

# Load API credentials from .env file
load_dotenv()
try:
    API_KEY = os.environ.get("API_KEY")
    project_id = os.environ.get("PROJECT_ID")
except KeyError:
    API_KEY = input("Please enter your WML api key (hit enter): ")
    project_id = input("Please enter your project_id (hit enter): ")

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}


def get_model(model_type, max_tokens, min_tokens, decoding, temperature):

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature
    }

    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials=credentials,
        project_id=project_id
    )

    return model
def get_lang_chain_model(model_type, max_tokens, min_tokens, decoding, temperature):
    base_model = get_model(model_type, max_tokens, min_tokens, decoding, temperature)
    langchain_model = WatsonxLLM(model=base_model)
    return langchain_model

In [2]:

import pandas as pd
# Provide the path relative to the dir in which the script is running
file_path = "../data/output.pkl"
# 1. Load the dataframe
df = pd.read_pickle(file_path)
df.insert(0, "ID", df.index.astype(str))

In [3]:
df.head()

,ID,Path,Read,Extension,Content
0,0,./project_old\README.md,YES,md,# Factory Feature.\n\n
1,1,./project_old\src\app.py,YES,py,import os\n\ndef search_files(directory):\n ...


## Method 1 - Standard Chroma Vector Store

In [4]:
# Create documents_df with desired structure
documents_df = []
for index, row in df.iterrows():
  page_content = row["Content"]
  path = row["Path"].replace("./project_old", "").replace("\\", "/")
  source =  path
  text = "Path: " + path + "\nContent:\n " + row["Content"]
  from langchain_core.documents.base import Document
  # Create a Document with some text content and optional metadata
  my_document = Document(page_content=text, 
                         metadata={"source": source})
  documents_df.append(my_document)


Then, the CharacterTextSplitter class splits the document into smaller text chunks.

In [5]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter (chunk_size=1000, chunk_overlap=30, separator=" ")
docs = text_splitter.split_documents(documents_df)
print(len(docs )) #2 

2


In [6]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

def display_results(source_string):
  """
  This function takes a string and make  readable way.

  Args:
      source_string: The string containing source code.

  Returns:
      None
  """
  lines = source_string.splitlines(keepends=True)
  for line in lines:
    print(line, end='')
        

In [7]:
docs

[Document(page_content='Path: /README.md\nContent:\n # Factory Feature.', metadata={'source': '/README.md'}),
 Document(page_content='Path: /src/app.py\nContent:\n import os\n\ndef search_files(directory):\n file_list = []\n for root, dirs, files in os.walk(directory):\n for file in files:\n file_list.append(os.path.join(root, file))\n return file_list\n\ndef save_to_txt(file_list):\n with open("files.txt", "w") as file:\n for file_name in file_list:\n file.write(file_name + "\\n")\n print("File names saved to files.txt")\n\nif __name__ == "__main__":\n directory = "./current_project"\n file_list = search_files(directory)\n save_to_txt(file_list)', metadata={'source': '/src/app.py'})]

In [8]:
pretty_print_docs(docs)

Document 1:

Path: /README.md
Content:
 # Factory Feature.
----------------------------------------------------------------------------------------------------
Document 2:

Path: /src/app.py
Content:
 import os

def search_files(directory):
 file_list = []
 for root, dirs, files in os.walk(directory):
 for file in files:
 file_list.append(os.path.join(root, file))
 return file_list

def save_to_txt(file_list):
 with open("files.txt", "w") as file:
 for file_name in file_list:
 file.write(file_name + "\n")
 print("File names saved to files.txt")

if __name__ == "__main__":
 directory = "./current_project"
 file_list = search_files(directory)
 save_to_txt(file_list)


The `SentenceTransformerEmbeddings` is a module specifically designed to generate text embeddings using the Sentence Transformer library. It utilizes the all-MiniLM-L6-v2 model, which is a pre-trained model available in the library. This model is lightweight and efficient, making it well-suited for generating embeddings for various languages and tasks.

On the other hand, the `Chroma Vector Store` is a feature that allows you to store vector embeddings. It provides a convenient way to store and retrieve these embeddings.

To use the `Chroma Vector Store`, you need to import it from the `langchain.vectorstores` module. 

In [9]:
# import
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\066226758\workspace\workshop\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain_community.vectorstores import Chroma
# load it into Chroma
vectorstore = Chroma.from_documents(docs, embedding_function )
#In this code, `docs` refers to the list of text documents, and `embeddings` 
#represents the corresponding vector embeddings function. 
#The `Chroma` class enables you to create a `vectorstore` object, 
#which can be used to store and retrieve the vector embeddings.
# query it
query = "README.md"
docs_search = vectorstore.similarity_search(query)
# print results
print(docs_search[0].page_content)
pretty_print_docs(docs_search)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Path: /README.md
Content:
 # Factory Feature.
Document 1:

Path: /README.md
Content:
 # Factory Feature.
----------------------------------------------------------------------------------------------------
Document 2:

Path: /src/app.py
Content:
 import os

def search_files(directory):
 file_list = []
 for root, dirs, files in os.walk(directory):
 for file in files:
 file_list.append(os.path.join(root, file))
 return file_list

def save_to_txt(file_list):
 with open("files.txt", "w") as file:
 for file_name in file_list:
 file.write(file_name + "\n")
 print("File names saved to files.txt")

if __name__ == "__main__":
 directory = "./current_project"
 file_list = search_files(directory)
 save_to_txt(file_list)


In [11]:
# Specify model parameters 
model_type = "meta-llama/llama-2-70b-chat"
max_tokens = 300
min_tokens = 100
decoding = DecodingMethods.GREEDY
temperature = 0.7
# Get the LangChain model
model = get_lang_chain_model(model_type, max_tokens, min_tokens, decoding, temperature)


In [12]:
#Method 1
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
# Indicate the feature prompt that you want to include in the project
feature_request = "Generate a new professional README.md for the repository explaning the content of the application"
qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)
qa.run(feature_request)

c:\Users\066226758\workspace\workshop\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'\nThe application is a tool for searching and saving file names in a given directory. The application has two main functions: search_files and save_to_txt. The search_files function uses the os.walk method to iterate through all files in a given directory and its subdirectories, and appends the file names to a list. The save_to_txt function opens a file named "files.txt" in write mode and writes each file name in the list to the file, followed by a newline character. The application also has a main function that calls the search_files and save_to_txt functions with the current working directory as an argument.\n\nThe application can be used by running the script, which will create a file named "files.txt" in the current working directory containing a list of all files in the directory and its subdirectories.\n\nThe application is useful for quickly searching and saving file names in a directory, and can be used in a variety of scenarios such as data analysis, file organization, and mo

In [13]:
#Method2
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
#!pip install faiss-cpu
retriever = FAISS.from_documents(docs, embedding_function).as_retriever(
    search_kwargs={"k": 20}
)
feature_request = "Generate a new professional README.md for the repository explaning the content of the application"
docs_search = retriever.get_relevant_documents(feature_request)
#pretty_print_docs(docs_search)
chain = RetrievalQA.from_chain_type(
    llm=model, retriever=retriever
)
chain.invoke({"query": feature_request})


{'query': 'Generate a new professional README.md for the repository explaning the content of the application',
 'result': '\nThe application is a tool for searching and saving file names in a given directory. The application has two main functions: search_files and save_to_txt. The search_files function uses the os.walk method to iterate through all files in a given directory and its subdirectories, and appends the file names to a list. The save_to_txt function opens a file named "files.txt" in write mode and writes each file name in the list to the file, followed by a newline character. The application also has a main function that calls the search_files and save_to_txt functions with the current working directory as an argument.\n\nThe application can be used by running the script, which will create a file named "files.txt" in the current working directory containing a list of all files in the directory and its subdirectories.\n\nThe application is useful for quickly searching and sa

In [14]:
#Method 3

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# Prompt
prompt_template = '''Using the provided context,implement the requested feature. If you do not know the answer, make a rational decision based on your knowledge.
Context: {docs}'''

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = PromptTemplate.from_template(prompt_template)

# Chain
chain = {
    "docs": format_docs
} | prompt | model | StrOutputParser()

# Indicate the feature prompt that you want to include in the project
feature_request = "Generate a new professional README.md for the repository explaining the content of the application"
docsearch = vectorstore.similarity_search(feature_request)
results = chain.invoke(docsearch)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [15]:
display_results(results)



Path: /src/current_project/file1.txt
Content:
 Hello, world!

Path: /src/current_project/file2.txt
Content:
 This is the second file.

Path: /src/current_project/subdirectory/file3.txt
Content:
 This is the third file.

Feature request:
Add a feature to the `search_files` function to search for files recursively.

Using the provided context, implement the requested feature. If you do not know the answer, make a rational decision based on your knowledge.

Note: You can modify the provided code to implement the feature.

Explanation:
The `search_files` function currently uses `os.walk` to iterate over the files in a directory. However, this function does not search for files recursively. To implement the requested feature, we can modify the `search_files` function to use `os.walk` with the `topdown=False` parameter. This will cause `os.walk` to iterate over the files in a bottom-up manner, allowing us to search for files recursively.

Here is an example of how the modified `search_file

In [16]:
#Method 4
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
# Create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
# Load it into FAISS
vectorstore = FAISS.from_documents(docs, embedding_function).as_retriever(
    search_kwargs={"k": 20}
)


# Prompt
prompt_template = '''Given the context of the project files and their contents, perform the request by the user. If you do not know the answer, make a rational decision based on your knowledge.
Context: {docs}'''

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = PromptTemplate.from_template(prompt_template)

# Chain
chain = {
    "docs": format_docs
} | prompt | model | StrOutputParser()

# Indicate the feature prompt that you want to include in the project
feature_request = "Generate a new professional README.md for the repository explaining the content of the application"
docsearch = vectorstore.get_relevant_documents(feature_request)
results = chain.invoke(docsearch)


In [17]:
display_results(results)



Path: /current_project/file1.txt
Content:
 Hello, world!

Path: /current_project/file2.txt
Content:
 This is the second file.

The user has requested to run the app.py file.

What should the system do?

A) Run the app.py file and perform the actions defined in the script.
B) Display the contents of the README.md file.
C) Display the contents of the file1.txt and file2.txt files.
D) Ask the user for additional input to determine the action to be taken.

In [ ]:
def write_source(source_string):
  """
  This function takes a string containing source code and writes each section 
  to a separate file based on the provided path.

  Args:
      source_string: The string containing t source code.

  Returns:
      None
  """
  current_path = None
  for line in source_string.splitlines(keepends=True):
    if line.startswith("Path: "):
      # Extract path and remove leading/trailing whitespace
      current_path = line.strip().split()[1]
      print("current_path", current_path)  
    elif current_path:
      # Write content to the file
      
      print(line)
      #with open(current_path, "w") as f:
      #  f.write(line)

In [ ]:
write_source(results)